## **Exploring linear regression: Predicting House Sale Prices**
By: Jennifer Kuo    |    Date: 1/8/19

## Inspirations:
This is my first kernel as I start to immerse myself in these Kaggle challenges. My goal here is to go through the workflow of preparing the data for training the model, and then to test a few different linear regression implementations. I hope you find this notebook helpful for exploration!

I've read through a few kernels for inspiration and insight, of which I'd like to highlight [Simple Linear Regression Models](https://www.kaggle.com/rbyron/simple-linear-regression-models) by Rocio Byron. She put together a great notebook with a clear framework.

## Outline:
1. Understand the problem and the data
2. Pre-processing (Data cleaning, normalizing the data, removing outliers) 
3. Feature Engineering
4. Train and Test
5. Compare Scores

## 1. Understanding the problem
We would like to predict the SalePrice of houses based on features provided in the Ames Housing data set. Let's take a look at our data.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
testID = test['Id']

data = pd.concat([train.drop('SalePrice', axis=1), test], keys=['train', 'test'])
data.drop(['Id'], axis=1, inplace=True)
data.head()

## 2. Pre-processing:

Data quality check:
1. Are the years of columns within a reasonable range (ie. less than 2018)?
2. Are the measurements within a reasonable range (ie. no negative numbers)?

In [ ]:
# Create list of years and metrics
years = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
metrics = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 
         'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

In [ ]:
data[years].max()

Looks like there is at least one entry in the GarageYrBlt column that is not an outlier, but rather a data entry error. The other year columns look fine. We will mask the error date with the date that the house was built. While this may not be correct, it is a better estimation as we don't have any other data to go off of. This is much more informative than keeping a spurious date that will be an outlier.

In [ ]:
mask = (data[years] > 2018).any(axis=1) # take any index with illogical year value
data[mask]['GarageYrBlt']

In [ ]:
data.loc[mask, 'GarageYrBlt'] = data[mask]['YearBuilt']
data[mask]['GarageYrBlt']

Moving on! We'll split our data into numerical and categorical data for ease of manipulation as we observe trends, cleanse, and impute missing values. 

In [ ]:
# Categorize features
numerical_feats = data.select_dtypes(include = ['float', 'integer']).columns
categorical_feats = data.select_dtypes(include = 'object').columns
numerical_feats

In [ ]:
# List all columns indicating a grade (refer to data dictionary)
grades = ['OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
          'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
data[grades].head()

In [ ]:
# By looking at the data dictionary, we can assign the literal grades to a numerical value that will be more informative
literal = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
num = [9, 7, 5, 3, 2]
# Create a dictionary
G = dict(zip(literal, num))
data[grades] = data[grades].replace(G)
data[grades].head()

There are still NA values in these columns so we will have to deal with the missing values later.

In [ ]:
# Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["MSSubClass", "MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating", "CentralAir", "GarageType", "MiscFeature", "SaleType", "SaleCondition"]

In [ ]:
# Explore which numerical columns should be converted to categorical
numerical_df = data.select_dtypes(include = ['float', 'integer'])
cat_cols = []
for col in numerical_df.columns:
    if col in nominal_features:
        cat_cols.append(col)
cat_cols

In [ ]:
# Adjust data type of variable MSSubClass to categorical
data['MSSubClass'] = data['MSSubClass'].astype('object', copy=False)

## Normalize the data

As we will be exploring different implementations of linear regression, we need to make sure that our data is normalized.  Let's first look at the distribution of the target variable, SalePrice.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
from scipy import stats
from scipy.stats import norm, skew

sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

The SalePrice variable is skewed right and non-normal. We will need to apply a log transformation to normalize the distribution.

In [ ]:
# Use the numpy fuction log1p which  applies log(1+x) to transform the target
price = np.log1p(train['SalePrice'])

# Check the new distribution 
sns.distplot(price , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(price)
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(price, plot=plt)
plt.show()

Better! Now let's take a look at the other numerical features and whether they are skewed.

In [ ]:
# Identify skewed continuous numerical features:
skewed_feats = data.loc['train'][metrics].apply(lambda x: x.skew(skipna=True)) #compute skewness
skewed_feats

In [ ]:
# Get index of skewed features
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
# Log transform skewed features
data[skewed_feats] = np.log1p(data[skewed_feats])

## Explore data and remove outliers

Let's plot some variables and see whether we can remove any obvious outliers.

In [ ]:
train['SalePrice'].head()

In [ ]:
# Add log-transformed SalePrice to train dataset
train['SalePrice'] = np.log1p(train['SalePrice'])
train['SalePrice'].head()

In [ ]:
# Observe correlation to determine which features to look at
print("Find most important features relative to target")
abs_corr_eff = train.corr()['SalePrice'].abs().sort_values(ascending = False)
print(abs_corr_eff)

In [ ]:
# Show columns with a correlation coefficient of larger than 0.5
abs_corr_eff[abs_corr_eff > 0.5]

In [ ]:
# Look into OverallQual column since highly correlated
sns.set_style("darkgrid")
fig, ax = plt.subplots()
ax.scatter(x = train['OverallQual'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('OverallQuality', fontsize=13)
plt.show()

We can confirm there is a positive correlation with median sale price and higher overall quality as indicated by our correlation of columns and this visualization. There may be a few outliers like the listing of the house with overall quality of 4 that sold well, but the evidence is not strong enough to warrant removing.

In [ ]:
# Look into OverallQual column since highly correlated
fig, ax = plt.subplots()
ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

There are two houses that list a GrLiv area of greater than 4000 that is selling much less than expected. We will remove these two listings as outliers so that they do not influence our model.

In [ ]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [ ]:
# Look into GarageCars column since highly correlated
fig, ax = plt.subplots()
ax.scatter(x = train['GarageCars'], y = train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GarageCars', fontsize=13)
plt.show()

There are few listings with 4-car garages but we can observe the overall positive correlation of higher sale price for more car space in the garage. The next highly correlated variable is also relating to garages. We will have to reduce our variables since these could introduce colinearity as they are both related to garages.

## Deal with missing values

In [ ]:
# Check null values of numerical features
nulls_num = pd.DataFrame(data[numerical_feats].isnull().sum().sort_values(ascending=False))
nulls_num.columns = ['Null Count']
nulls_num = nulls_num[nulls_num['Null Count'] > 0]
nulls_num

In [ ]:
#Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
data["LotFrontage"] = data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

By reading the documentation, we can assume that the missing Basement, Garage, Masonry variables are missing because they do not exist.

In [ ]:
# Fill in missing values of numerical columns with 0
data[numerical_feats] = data[numerical_feats].fillna(0)
nulls_num = pd.DataFrame(data[numerical_feats].isnull().sum().sort_values(ascending=False))
nulls_num.columns = ['Null Count']
nulls_num = nulls_num[nulls_num['Null Count'] > 0]
nulls_num

In [ ]:
# Let's look at PoolQC values to check distribution
data['PoolQC'].value_counts()

If we only have real data points for 10 listings, is this even a useful feature for our model?

In [ ]:
# Check null values of categorical columns
nulls_cat = pd.DataFrame(data[categorical_feats].isnull().sum().sort_values(ascending=False))
nulls_cat.columns = ['Null Count']
nulls_cat = nulls_cat[nulls_cat['Null Count'] > 0]
nulls_cat

In [ ]:
# Fill in missing value with NA = typical
data["Functional"] = data["Functional"].fillna("Typ")

# Fill in missing value with most common value
data['MSZoning'] = data['MSZoning'].fillna(data['MSZoning'].mode()[0])

# Fill in missing value with most common value
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])

# Fill in missing value with most common value
data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])

# Substitute most common string for missing values
data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])

# Fill in most common sale type for missing value
data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])

In [ ]:
# Check null values of categorical columns
nulls_cat = pd.DataFrame(data[categorical_feats].isnull().sum().sort_values(ascending=False))
nulls_cat.columns = ['Null Count']
nulls_cat = nulls_cat[nulls_cat['Null Count'] > 0]
nulls_cat

In [ ]:
# Fill in missing values of categorical columns with '0'
data[categorical_feats] = data[categorical_feats].fillna('0')
nulls_cat = pd.DataFrame(data[categorical_feats].isnull().sum().sort_values(ascending=False))
nulls_cat.columns = ['Null Count']
nulls_cat = nulls_cat[nulls_cat['Null Count'] > 0]
nulls_cat

## 3.  Feature engineering

In [ ]:
# Adding total sqfootage feature 
data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

In [ ]:
# Engineer new features for years before sale, years since remodel
years_sold = data['YrSold'] - data['YearBuilt']  
years_remodeled = data['YrSold'] - data['YearRemodAdd'] 
data['Years Before Sale'] = years_sold
data['Years Since Remodel'] = years_remodeled
data.head()

## Group categorical data with few samples
Some categories of the categorical features are so unrrepresented in the dataset that drawing conclusions from them would lead to a noisy result. Instead, we will group those in one single category.

In [ ]:
train[categorical_feats].describe()

In [ ]:
# Drop columns with less than 0.6 correlation with SalePrice
feats_to_drop = abs_corr_eff[abs_corr_eff < 0.6].index
for i in feats_to_drop:
    if i in data.columns:
        data = data.drop(i, axis=1)
    else:
        pass

In [ ]:
# Drop utilities column since no information added for predictive modeling
feats_to_drop_1 = ['Utilities', 'PoolQC', 'Alley', 'Street']
for i in feats_to_drop_1:
    if i in data.columns:
        data = data.drop(i, axis=1)
    else:
        pass
# Drop old year columns that were replaced with 'Years Before Sale' and 'Years Since Remodel' feature
feats_to_drop_2 = ['YearBuilt', 'YearRemodAdd', 'YrSold']
for i in feats_to_drop_2:
    if i in data.columns:
        data = data.drop(i, axis=1)
    else:
        pass
# Drop old year columns that were replaced with 'TotalSF' feature
feats_to_drop_3 = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
for i in feats_to_drop_3:
    if i in data.columns:
        data = data.drop(i, axis=1)
    else:
        pass
# Drop Neighborhood column because too many unique values
feats_to_drop_4 = ['Neighborhood']
for i in feats_to_drop_4:
    if i in data.columns:
        data = data.drop(i, axis=1)
    else:
        pass

In [ ]:
data.columns

In [ ]:
cat_cols = data.select_dtypes(include = 'object').columns
cat_cols

In [ ]:
data.shape

In [ ]:
finaldata = pd.get_dummies(data)

In [ ]:
finaldata.shape

It would be cleaner (and less overfitting) to remove the '0' option as an encoding.

In [ ]:
bsmt = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
        'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'BsmtFullBath',
        'BsmtHalfBath', 
        'TotalBsmtSF']
fire = ['Fireplaces', 'FireplaceQu']
garage = ['GarageQual', 'GarageCond', 'GarageType', 'GarageFinish', 'GarageCars', 
          'GarageArea', 'GarageYrBlt']
masn = ['MasVnrType', 'MasVnrArea']
others = ['Alley', 'Fence', 'PoolQC', 'MiscFeature']

black_list = bsmt + fire + garage + masn + others
for feat in finaldata.columns:
    if ('_0' in feat) and (feat.split("_")[0] in black_list):
        finaldata.drop(feat, axis=1, inplace=True)

In [ ]:
finaldata.shape

## 4. Split data, Train Model and Test

In [ ]:
# Training/testing sets
X_test = finaldata.loc['test']
X_train = finaldata.loc['train']

y_train = price

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create linear regression object
LR = LinearRegression()

# Train the model using the training sets
LR.fit(X_train, y_train)

In [ ]:
# Top influencers
maxcoef = np.argsort(-np.abs(LR.coef_))
coef = LR.coef_[maxcoef]
for i in range(0, 5):
    print("{:.<025} {:< 010.4e}".format(finaldata.columns[maxcoef[i]], coef[i]))

When we think about buying a house, the Roof Materials and MSZoning probably aren't the first thing we think about. The coefficient value represents the mean change in the response given a one unit change in the predictor. Would these features really have such a large impact? Our model is likely overfitted. Let's try using an L1 regularization technique, Lasso. This technique will perform feature selection for us and reduce the magnitude of the coefficients. I found this [article](https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression) very helpful.

In [ ]:
from sklearn.linear_model import LassoCV

# Create linear regression object
Ls = LassoCV()

# Train the model using the training sets
Ls.fit(X_train, y_train)

In [ ]:
maxcoef = np.argsort(-np.abs(Ls.coef_))
coef = Ls.coef_[maxcoef]
for i in range(0, 5):
    print("{:.<025} {:< 010.4e}".format(finaldata.columns[maxcoef[i]], coef[i]))

The features that have the largest coefficient are GrLiveArea and OverallQual, which make more sense when we think about the qualities we consider when buying a home. Now, let's experiment with an L2 regularization technique, Ridge regression.

In [ ]:
from sklearn.linear_model import RidgeCV

# Create linear regression object
Rr = RidgeCV()

# Train the model using the training sets
Rr.fit(X_train, y_train)

In [ ]:
maxcoef = np.argsort(-np.abs(Rr.coef_))
coef = Rr.coef_[maxcoef]
for i in range(0, 5):
    print("{:.<025} {:< 010.4e}".format(finaldata.columns[maxcoef[i]], coef[i]))

The coefficients are consistent with the Lasso regression. We can also experiment with Elastic Net, which implements both the L1 and L2 penalty term by combining them algebraically.

In [ ]:
from sklearn.linear_model import ElasticNetCV

# Create linear regression object
EN = ElasticNetCV(l1_ratio=np.linspace(0.1, 1.0, 5)) # we are essentially smashing most of the Rr model here

# Train the model using the training sets
train_EN = EN.fit(X_train, y_train)

In [ ]:
maxcoef = np.argsort(-np.abs(EN.coef_))
coef = EN.coef_[maxcoef]
for i in range(0, 5):
    print("{:.<025} {:< 010.4e}".format(finaldata.columns[maxcoef[i]], coef[i]))

## 5. Compare Scores 

In [ ]:
from sklearn.model_selection import cross_val_score

model = [Ls, Rr, EN]
M = len(model)
score = np.empty((M, 5))
for i in range(0, M):
    score[i, :] = cross_val_score(model[i], X_train, y_train, cv=5)

In [ ]:
# print out all scores
score

In [ ]:
# get the average of scores for each model
print(score.mean(axis=1))

## Prepare submission file

In [ ]:
# My model based on ridge regression performed the best through cross-validation
submit = pd.DataFrame({'Id': testID, 'SalePrice': np.exp(Rr.predict(X_test))})
submit.to_csv('submission.csv', index=False)

## Future improvements

* I would revisit the correlation coefficients and experiment with the cutoff threshold for the  variables I keep. 
* There are a lot of new features created from one-hot-encoding the categorical data. However, it might be better to bin the values into groups so that instead of 5 new categories being created, only 2 categories are or engineer new features out of combining categories.

If you read this and found it useful or have ideas for improvement, please share!